# \[PD 4\] SVM 

Nie robię EDA, bo nie jest to celem zadania. Skupię się tylko na strojeniu parametrów SVM.

1. Na początek wczytuje dane **apartments**  i patrzę czy wszystko jest z nimi git.
2. Dopasowuję dwa modele SVR z rbf i z wielomianami, sprawdzam jakośc regresji za pomocą kroswalidajci 5 krotnej i RMSE.
3. Optymalizuję parametry i sprawdzam czy wynik się zmienił.
4. Skaluję dane, stroję hiperparametry i ponownie sprawdzam wynik.
5. Podsumowuję w dwóch zdaniach.

To samo robię z drugim zbiorem danych to jest z **pima**. Z tym że tutaj sprawdzam jakosć klasyfikacji za pomocą AUC.

Podsumowuję krótko cały ten ambaras.

In [49]:
# ŁADOWANIE NAJWAŻNIEJSZYCH PAKIETÓW
import pandas as pd
import numpy as np
import sklearn
import scipy.stats

In [5]:
# WCZYTYWANIE DANYCH
# 1. apartments z R-owego pakietu DALEX
apartments=pd.read_csv("C:\\Users\\huber\\Dropbox\\PW\\2 SEMESTR\\Wstep do uczenia maszynowego\\apartments.csv")

In [8]:
apartments.head()

,Unnamed: 0,m2.price,construction.year,surface,floor,no.rooms,district
0,1,5897,1953,25,3,1,Srodmiescie
1,2,1818,1992,143,9,5,Bielany
2,3,3643,1937,56,1,2,Praga
3,4,3517,1995,93,7,3,Ochota
4,5,3013,1992,144,6,5,Mokotow


In [9]:
apartments=apartments.drop(columns="Unnamed: 0")

In [10]:
apartments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   m2.price           1000 non-null   int64 
 1   construction.year  1000 non-null   int64 
 2   surface            1000 non-null   int64 
 3   floor              1000 non-null   int64 
 4   no.rooms           1000 non-null   int64 
 5   district           1000 non-null   object
dtypes: int64(5), object(1)
memory usage: 47.0+ KB


In [33]:
apartments.district.unique()

array(['Srodmiescie', 'Bielany', 'Praga', 'Ochota', 'Mokotow', 'Ursus',
       'Zoliborz', 'Wola', 'Bemowo', 'Ursynow'], dtype=object)

In [11]:
apartments.describe()

,m2.price,construction.year,surface,floor,no.rooms
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,3487.019000,1964.823000,85.590000,5.623000,3.360000
std,906.691651,25.831511,37.874799,2.899876,1.381415
min,1607.000000,1920.000000,20.000000,1.000000,1.000000
25%,2857.000000,1943.000000,53.000000,3.000000,2.000000
50%,3386.000000,1965.000000,85.500000,6.000000,3.000000
75%,4018.250000,1988.000000,118.000000,8.000000,4.000000
max,6595.000000,2010.000000,150.000000,10.000000,6.000000


Robię ONE HOT encoding dla dystryktu, wyrzucam jeden z nich, żeby pozbyć się współliniowości. 

In [135]:
# Onehotencoding
from category_encoders.one_hot import OneHotEncoder
OneHot=OneHotEncoder()
df=OneHot.fit_transform(apartments.district)
apartments_df=pd.concat([apartments.reset_index(drop=True), df], axis=1)
apartments_df=apartments_df.drop(columns="district")

C:\Users\huber\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [142]:
apartments_df

,m2.price,construction.year,surface,floor,no.rooms,district_1,district_2,district_3,district_4,district_5,district_6,district_7,district_8,district_9,district_10
0,5897,1953,25,3,1,1,0,0,0,0,0,0,0,0,0
1,1818,1992,143,9,5,0,1,0,0,0,0,0,0,0,0
2,3643,1937,56,1,2,0,0,1,0,0,0,0,0,0,0
3,3517,1995,93,7,3,0,0,0,1,0,0,0,0,0,0
4,3013,1992,144,6,5,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,6355,1921,44,2,2,1,0,0,0,0,0,0,0,0,0
996,3422,1921,48,10,2,0,0,0,0,0,0,0,0,1,0
997,3098,1980,85,3,3,0,0,0,0,0,0,0,0,1,0
998,4192,1942,36,7,1,0,0,0,0,0,0,1,0,0,0


In [152]:
x

,construction.year,surface,floor,no.rooms,district_1,district_2,district_3,district_4,district_5,district_6,district_7,district_8,district_9
0,1953,25,3,1,1,0,0,0,0,0,0,0,0
1,1992,143,9,5,0,1,0,0,0,0,0,0,0
2,1937,56,1,2,0,0,1,0,0,0,0,0,0
3,1995,93,7,3,0,0,0,1,0,0,0,0,0
4,1992,144,6,5,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1921,44,2,2,1,0,0,0,0,0,0,0,0
996,1921,48,10,2,0,0,0,0,0,0,0,0,1
997,1980,85,3,3,0,0,0,0,0,0,0,0,1
998,1942,36,7,1,0,0,0,0,0,0,1,0,0


In [154]:
from sklearn.svm import SVR 
from sklearn.model_selection import cross_val_score
SVR_rbf=SVR(kernel='rbf')
SVR_pol=SVR(kernel='poly')
x=apartments_df.iloc[:,1:14]
y=apartments_df.iloc[:,0]
results=cross_val_score(SVR_rbf, x, y, cv=5, scoring='neg_root_mean_squared_error')
pd.DataFrame(results).describe()

,0
count,5.000000
mean,-910.563187
std,71.342977
min,-1019.743427
25%,-918.619237
50%,-906.007464
75%,-885.883974
max,-822.561832


In [156]:
results=cross_val_score(SVR_pol, x, y, cv=5, scoring="neg_root_mean_squared_error")
pd.DataFrame(results).describe()

,0
count,5.000000
mean,-910.449869
std,71.337987
min,-1019.616494
25%,-918.507871
50%,-905.910925
75%,-885.765491
max,-822.448563


Teraz wybieram w miarę sensowne rozkłady a priori, dla dyskretnych parametrów dyskretne, dla ciągłych ciągłe. Staram się dopasować parametr położenia rozkładu do często stosowanych wartości parametru znalezionych w internecie. 

In [177]:
from sklearn.model_selection import RandomizedSearchCV
random = RandomizedSearchCV(SVR_pol,param_distributions={'C': scipy.stats.expon(scale=100000000), 'degree': scipy.stats.poisson(3),
  'kernel': ['poly']}, cv=5, scoring='neg_root_mean_squared_error')
search=random.fit(x, y)
print('Best Score: %s' % search.best_score_)
print('Best Hyperparameters: %s' % search.best_params_)

Best Score: -305.08662104561483
Best Hyperparameters: {'C': 211139470.98934335, 'degree': 3, 'kernel': 'poly'}


In [173]:
random = RandomizedSearchCV(SVR_rbf,param_distributions={'C': scipy.stats.expon(scale=1000000), 'gamma': scipy.stats.expon(scale=0.00001),
  'kernel': ['rbf']}, cv=5, scoring='neg_root_mean_squared_error')
search=random.fit(x, y)
print('Best Score: %s' % search.best_score_)
print('Best Hyperparameters: %s' % search.best_params_)

Best Score: -143.3887603620954
Best Hyperparameters: {'C': 5131899.561300474, 'gamma': 2.6173476822726904e-05, 'kernel': 'rbf'}


Wydaje się, że jądro **rbf** duże lepiej się spisuje. Na pewno wyniki są dużo lepsze niż w wyjściowej wersji z domyślnymi parametrami.

In [178]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_SS=scaler.fit_transform(x)

In [183]:
from sklearn.model_selection import RandomizedSearchCV
random = RandomizedSearchCV(SVR_pol,param_distributions={'C': scipy.stats.expon(scale=100000), 'degree': scipy.stats.poisson(5),
  'kernel': ['poly']}, cv=5, scoring='neg_root_mean_squared_error')
search=random.fit(x_SS, y)
print('Best Score: %s' % search.best_score_)
print('Best Hyperparameters: %s' % search.best_params_)

Best Score: -215.81521898042166
Best Hyperparameters: {'C': 180386.97815691133, 'degree': 4, 'kernel': 'poly'}


In [191]:
random = RandomizedSearchCV(SVR_rbf,param_distributions={'C': scipy.stats.expon(scale=100000), 'gamma': scipy.stats.expon(scale=0.01),
  'kernel': ['rbf']}, cv=5, scoring='neg_root_mean_squared_error')
search=random.fit(x_SS, y)
print('Best Score: %s' % search.best_score_)
print('Best Hyperparameters: %s' % search.best_params_)

Best Score: -146.73440798852374
Best Hyperparameters: {'C': 83973.58128657035, 'gamma': 0.0035106150998256347, 'kernel': 'rbf'}


Wnioski:
1. Optymalizacja kroswalidacyjzna hiperparametrów z użyciem rozkładów a priori daje lepsze wyniki niż domyślna wersja.
2. Wystandaryzowanie zmiennych odejmując średnią i dzieląc przez odchylenie standardowe czasem poprawia wynik, ale ie zawsze znacząco. Raczej standaryzowanie powinno przynosić lepsze wyniki, a ich nie pogarszać, oczywiście to zależy od przypadku.

## 2. Dowolny, wybrany przez siebie zbiór danych. Dane pochodzą z https://sci2s.ugr.es/keel/dataset.php?cod=21.  
 No jest 7 zmiennych numerycznych, ale chyba też będzie git zobaczyć jak się SVM sprawuje, zwłaszcza że ósma zmienna jest binarna i sprawdzam tutaj dla odmiany klasyfikację, a nie regresję

In [13]:
pima=pd.read_csv("C:\\Users\\huber\\Dropbox\\PW\\2 SEMESTR\\Wstep do uczenia maszynowego\\pima.dat")

In [14]:
pima.head()

,Preg,Plas,Pres,Skin,Insu,Mass,Pedi,Age,Class
0,14,175,62,30,0,33.6,0.212,38,tested_positive
1,4,146,78,0,0,38.5,0.520,67,tested_positive
2,15,136,70,32,110,37.1,0.153,43,tested_positive
3,3,107,62,13,48,22.9,0.678,23,tested_positive
4,3,169,74,19,125,29.9,0.268,31,tested_positive


In [15]:
pima.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Preg    768 non-null    int64  
 1   Plas    768 non-null    int64  
 2   Pres    768 non-null    int64  
 3   Skin    768 non-null    int64  
 4   Insu    768 non-null    int64  
 5   Mass    768 non-null    float64
 6   Pedi    768 non-null    float64
 7   Age     768 non-null    int64  
 8   Class   768 non-null    object 
dtypes: float64(2), int64(6), object(1)
memory usage: 54.1+ KB


In [21]:
pima.loc[list(np.where(pima.Class=="tested_positive"))[0],"Class"]=1
pima.loc[list(np.where(pima.Class=="tested_negative"))[0],"Class"]=0

In [30]:
pima.Class=pima.Class.astype(int)

In [31]:
pima.describe()

,Preg,Plas,Pres,Skin,Insu,Mass,Pedi,Age,Class
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


### Dopasowywanie

In [44]:
from sklearn.svm import SVC 
from sklearn.model_selection import cross_val_score
SVM_rbf=SVC(kernel='rbf')
SVM_pol=SVC(kernel='poly')
X=pima.iloc[:,0:7]
y=pima.iloc[:,8]
results=cross_val_score(SVM_rbf, X, y, cv=5, scoring="roc_auc")
pd.DataFrame(results).describe()

,0
count,5.000000
mean,0.805005
std,0.053500
min,0.739623
25%,0.761698
50%,0.818333
75%,0.835741
max,0.869630


In [45]:
results=cross_val_score(SVM_pol, X, y, cv=5, scoring="roc_auc")
pd.DataFrame(results).describe()

,0
count,5.000000
mean,0.814638
std,0.045515
min,0.767170
25%,0.775094
50%,0.810370
75%,0.848333
max,0.872222


Na domyślnych parametrach lepiej nieznacznie radzi sobie patrząc po medianie **rbf**, ale minimum i maximum wyższe ma wersja **wielomianowa**

### Strojenie hiperparametrów

In [47]:
from sklearn.model_selection import RandomizedSearchCV

In [113]:
random = RandomizedSearchCV(SVM,param_distributions={'C': scipy.stats.expon(scale=1000), 'gamma': scipy.stats.expon(scale=0.0001),
  'kernel': ['rbf']}, cv=5, scoring='roc_auc')
search=random.fit(X, y)

In [114]:
print('Best Score: %s' % search.best_score_)
print('Best Hyperparameters: %s' % search.best_params_)

Best Score: 0.8143542976939202
Best Hyperparameters: {'C': 1593.3257976674756, 'gamma': 7.497602818416913e-06, 'kernel': 'rbf'}


In [115]:
from sklearn.model_selection import RandomizedSearchCV
SVM=SVC()
random = RandomizedSearchCV(SVM,param_distributions={'C': scipy.stats.expon(scale=1000), 'degree': scipy.stats.poisson(3),
  'kernel': ['poly']}, cv=5, scoring='roc_auc')
search=random.fit(X, y)

In [116]:
print('Best Score: %s' % search.best_score_)
print('Best Hyperparameters: %s' % search.best_params_)

Best Score: 0.8186624737945494
Best Hyperparameters: {'C': 19.01884783293252, 'degree': 2, 'kernel': 'poly'}


Jakoś dużo lepiej się te wyniki nie poprawiły. Może powinien zostać inny rozkład zaproponowany. Znowu średni wynik lepszy w **poly** niż w **rbf**.

### Test wyników w zależności od skalowania danych
eraz standaryzuje kolumny w ramce danych i jeszcze raz stosuję random grid searcha.

In [121]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_SS=scaler.fit_transform(X)

In [126]:
from sklearn.model_selection import RandomizedSearchCV
SVM=SVC()
random = RandomizedSearchCV(SVM,param_distributions={'C': scipy.stats.expon(scale=1000), 'degree': scipy.stats.poisson(3),
  'kernel': ['poly']}, cv=5, scoring='roc_auc')
search=random.fit(X_SS, y)
print('Best Score: %s' % search.best_score_)
print('Best Hyperparameters: %s' % search.best_params_)

Best Score: 0.8296512928022363
Best Hyperparameters: {'C': 202.21878696138026, 'degree': 1, 'kernel': 'poly'}


In [132]:
random = RandomizedSearchCV(SVM,param_distributions={'C': scipy.stats.expon(scale=1000), 'gamma': scipy.stats.expon(scale=0.00001),
  'kernel': ['rbf']}, cv=5, scoring='roc_auc')
search=random.fit(X_SS, y)
print('Best Score: %s' % search.best_score_)
print('Best Hyperparameters: %s' % search.best_params_)

Best Score: 0.8303724668064291
Best Hyperparameters: {'C': 2261.249287211476, 'gamma': 6.03552438050629e-06, 'kernel': 'rbf'}


Spodziewałem się, że wyniki się dużo bardziej poprawią. Tak zresztą wskazuje teoria, że klasy będą lepiej separowalne po wystandaryzowaniu danych. Trochę się poprawiły niemniej jednak. Poprawa nie jest tak łatwo widoczna jak w przypadku regresji.

## Podsumowanie
SVM i SVR to potężne metody regresji i klasyfikacji. Ich wadą jest trudna interpretowalność, ich zaletą dobra skuteczność. Kolejną ich wadą jest duża wrażliwość na przeprocesowanie dancy, to jest wyskalowanie ich w odpowiedni sposób, a także ważność wyboru hiperparametrów. Na sczęście dla badaczy i analityków dostępne są przystępne metody optymalizacji hiperparametrów np w Python sklearn.